In [1]:
import json
from scripts import step_1_generate_roi, step_2_deploy, step_3_detect_candidates, step_4_query_patch, step_5_download_candidates

In [2]:
%load_ext autoreload
%autoreload 2

In [35]:
template = {
    "roi": {
        "name": "ROI Name",
        "region": False,
    },
    "dates": {
        "start": "2021-01-01",
        "end": "2022-07-01"
    },
    "data": {
        "mosaic_period": 3,
        "mosaic_method": "min",
        "spectrogram_interval": 2
    },
    "pixel": {
        "version": "0.0.11",
        "name": "spectrogram_v0.0.11_2021-07-13",
    },
    "patch": {
        "version": "weak_labels_3.0",
        "name": "weak_labels_3.0_batchnorm",
        "stride": 8,
    },
    "candidate_detect": {
        "pred_threshold": 0.3,
        "min_sigma": 5.0
    },
    "intersect": {
        "patch_threshold": 0.3,
    },
    "run_local": False
}

In [30]:
country_list = ['Korea North', 'Korea South', 'New Caledonia']

In [26]:
for country in country_list:
    config = template.copy()
    config['roi']['name'] = country.lower().replace(' ', '_')
    #config = json.dumps(config)
    with open(f"../pipeline/{country.lower().replace(' ', '_')}_config.json", 'w') as f:
        json.dump(config, f, indent=4)
    

In [27]:
roi_failures = {}
for country in country_list:
    print(country)
    try:
        step_1_generate_roi.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        roi_failures[country] = e
        print(f"{country} failed: {e}")
        continue

Korea, North
Approx DL Time: 3.16 hours
Number of DL Tiles to process: 1582
Wrote dlkeys to ../data/boundaries/dlkeys/korea,_north_pop_0.25_dlkeys.txt
Korea, South
Approx DL Time: 2.67 hours
Number of DL Tiles to process: 1333
Wrote dlkeys to ../data/boundaries/dlkeys/korea,_south_pop_0.25_dlkeys.txt
New Caledonia
Approx DL Time: 0.13 hours
Number of DL Tiles to process: 64
Wrote dlkeys to ../data/boundaries/dlkeys/new_caledonia_pop_0.25_dlkeys.txt


In [31]:
from scripts import step_2_deploy
detect_failures = {}
for country in [c for c in country_list if c not in roi_failures.keys() and c != 'Samoa']:
    print(country)
    try:
        step_2_deploy.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        detect_failures[country] = e
        print(f"{country} failed: {e}")
        continue

Korea North
Read 1582 dlkeys from ../data/boundaries/dlkeys/korea_north_pop_0.25_dlkeys.txt
Estimated cost for run: $25.60
Got patch product earthrise:korea_north_v0.0.11_2021-01-01_2022-07-01
Model spectrogram_v0.0.11_2021-07-13 found in DLStorage.
Model weak_labels_3.0_batchnorm found in DLStorage.
Creating product earthrise:korea_north_patch_weak_labels_3.0_2021-01-01_2022-07-01_stride_8_patches


100%|██████████| 1582/1582 [03:01<00:00,  8.70it/s]


Korea South
Read 1333 dlkeys from ../data/boundaries/dlkeys/korea_south_pop_0.25_dlkeys.txt
Estimated cost for run: $21.57
Got patch product earthrise:korea_south_v0.0.11_2021-01-01_2022-07-01
Model spectrogram_v0.0.11_2021-07-13 found in DLStorage.
Model weak_labels_3.0_batchnorm found in DLStorage.
Creating product earthrise:korea_south_patch_weak_labels_3.0_2021-01-01_2022-07-01_stride_8_patches


100%|██████████| 1333/1333 [02:42<00:00,  8.21it/s]


New Caledonia
Existing product found.
Deleting db97e4b985ef52fa21d7e83dd688047fada1a0f2:earthrise:new_caledonia_patch_weak_labels_3.0_2021-01-01_2022-07-01_stride_8_patches
Read 64 dlkeys from ../data/boundaries/dlkeys/new_caledonia_pop_0.25_dlkeys.txt
Estimated cost for run: $1.04
Got patch product earthrise:new_caledonia_v0.0.11_2021-01-01_2022-07-01
Model spectrogram_v0.0.11_2021-07-13 found in DLStorage.
Model weak_labels_3.0_batchnorm found in DLStorage.
Creating product earthrise:new_caledonia_patch_weak_labels_3.0_2021-01-01_2022-07-01_stride_8_patches


100%|██████████| 64/64 [00:08<00:00,  7.23it/s]


In [32]:
candidate_failures = {}
for country in [c for c in country_list if c not in detect_failures.keys() and c not in roi_failures.keys()]:
    print(country)
    try:
        step_3_detect_candidates.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        candidate_failures[country] = e
        print(f"{country} failed: {e}")
        continue

# retry failed countries
for country in [c for c in candidate_failures.keys()]:
    print(country)
    try:
        step_3_detect_candidates.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
        del(candidate_failures[country])
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        candidate_failures[country] = e
        print(f"{country} failed: {e}")
        continue

Korea North
Creating product earthrise:korea_north_v0.0.11_2021-01-01_2022-07-01_blobs_thresh_0.6_min-sigma_5.0_area-thresh_0.0025_band-median


100%|██████████| 1582/1582 [03:12<00:00,  8.20it/s]


Korea South
Creating product earthrise:korea_south_v0.0.11_2021-01-01_2022-07-01_blobs_thresh_0.6_min-sigma_5.0_area-thresh_0.0025_band-median


100%|██████████| 1333/1333 [02:32<00:00,  8.74it/s]


New Caledonia
Creating product earthrise:new_caledonia_v0.0.11_2021-01-01_2022-07-01_blobs_thresh_0.6_min-sigma_5.0_area-thresh_0.0025_band-median


100%|██████████| 50/50 [00:06<00:00,  7.79it/s]


In [33]:
query_failures = {}
for country in country_list:
    print(country)
    config_path = f"../pipeline/{country.lower().replace(' ', '_')}_config.json"
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        # check if candidate detect product exists
        print(config['candidate_detect']['product_id'])
        step_4_query_patch.main(config_path=config_path)
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        query_failures[country] = e
        print(f"{country} failed: {e}")
        continue

Korea North
korea_north_v0.0.11_2021-01-01_2022-07-01_blobs_thresh_0.6_min-sigma_5.0_area-thresh_0.0025_band-median
Korea South
korea_south_v0.0.11_2021-01-01_2022-07-01_blobs_thresh_0.6_min-sigma_5.0_area-thresh_0.0025_band-median
New Caledonia
new_caledonia_v0.0.11_2021-01-01_2022-07-01_blobs_thresh_0.6_min-sigma_5.0_area-thresh_0.0025_band-median


In [34]:
download_failures = {}
for country in country_list:
    print("Downloading", country)
    config_path = f"../pipeline/{country.lower().replace(' ', '_')}_config.json"
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        # check if candidate detect product exists
        product_id = config['intersect']['product_id']
        step_5_download_candidates.main(config_path=config_path)
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        download_failures[country] = e
        print(f"{country} failed: {e}")
        continue

No candidates found for 0.0.11
Korea North failed: list index out of range


In [ ]:
import os
import geopandas as gpd
import pandas as pd
combined_df = gpd.GeoDataFrame()
for f in os.listdir('../data/model_outputs/candidate_sites/0.0.11/'):
    if f.startswith('intersect_'):
        gdf = gpd.read_file(f'../data/model_outputs/candidate_sites/0.0.11/{f}')
        combined_df = gpd.GeoDataFrame(pd.concat([combined_df, gdf]))

run_title = 'caledonia_and_korea'
combined_df.to_file(f'../data/model_outputs/candidate_sites/0.0.11/{run_title}.geojson', driver='GeoJSON')